In [1]:
from nfl_data_py import import_players, import_weekly_data

# Fetch player stats data
player_stats = import_weekly_data([2022])

# Fetch player metadata
players = import_players()

Downcasting floats.


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Combine player stats and player metadata into a single DataFrame
df = pd.merge(player_stats, players, left_on='player_id', right_on='gsis_id', how='left')

# Sort the dataframe by player_id and week
df = df.sort_values(['player_id', 'week'])

# Create the target variable (next week's passing yards)
df['next_week_passing_yards'] = df.groupby('player_id')['passing_yards'].shift(-1)

# Remove the last week for each player (as we don't have next week's data for it)
df = df.groupby('player_id').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)

# Encode categorical features
label_encoder = LabelEncoder()
df['position_x'] = label_encoder.fit_transform(df['position_x'])
df['recent_team'] = label_encoder.fit_transform(df['recent_team'])

# Scale numeric features
scaler = StandardScaler()
df[['passing_yards', 'rushing_yards', 'receiving_yards']] = scaler.fit_transform(df[['passing_yards', 'rushing_yards', 'receiving_yards']])

# Remove rows with NaN values
df = df.dropna(subset=['next_week_passing_yards', 'passing_yards', 'rushing_yards', 'receiving_yards'])

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_11749/3283583966.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('player_id').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)


In [3]:
X = df[['position_x', 'recent_team', 'passing_yards', 'rushing_yards', 'receiving_yards']]
y = df['next_week_passing_yards']

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# Define the model architecture
model = Sequential()
model.add(Input(shape=(5,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2)

2024-08-22 17:11:29.790152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-22 17:11:29.844562: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-22 17:11:29.859872: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-22 17:11:29.956605: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-22 17:11:30.996934: W tensorflow/compiler/tf2

Epoch 1/100


I0000 00:00:1724368293.552162   12425 service.cc:146] XLA service 0x77592c008ff0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724368293.552188   12425 service.cc:154]   StreamExecutor device (0): Quadro T1000, Compute Capability 7.5
2024-08-22 17:11:33.573932: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-22 17:11:33.679797: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


121/126 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step - loss: 6606.0762

I0000 00:00:1724368294.378004   12425 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


126/126 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 6576.0400 - val_loss: 3924.3882
Epoch 2/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5347.1914 - val_loss: 2396.7290
Epoch 3/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2561.3137 - val_loss: 1363.2777
Epoch 4/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1094.4851 - val_loss: 1325.2102
Epoch 5/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1072.8134 - val_loss: 1280.3845
Epoch 6/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 880.3699 - val_loss: 1305.9402
Epoch 7/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 947.9210 - val_loss: 1237.9618
Epoch 8/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 938.7597 - val_loss: 1413.7666
Epoch 9/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1021.2940 - val_loss: 1208.7161
Epoch 10/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 943.0172 - val_loss: 1190.0219
Epoch 11/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 818.0247 - val_loss: 11

In [5]:
import numpy as np

# Select a player (in this case, the first one in the DataFrame)
player_name = "Patrick Mahomes"  # Replace with the name of the player you're interested in
player_stats = df[df['player_display_name'] == player_name].iloc[0]

# Print player information
print(f"Player: {player_stats['player_name']} (ID: {player_stats['player_id']})")
print(f"Position: {player_stats['position_x']}")
print(f"Team: {player_stats['recent_team']}")
print(f"Current week stats:")
print(f"  Passing yards: {player_stats['passing_yards']}")
print(f"  Rushing yards: {player_stats['rushing_yards']}")
print(f"  Receiving yards: {player_stats['receiving_yards']}")

input_data = np.array([[
    player_stats['position_x'],
    player_stats['recent_team'],
    player_stats['passing_yards'],
    player_stats['rushing_yards'],
    player_stats['receiving_yards']
]])

next_week_passing_yards = None

try:
    prediction = model.predict(input_data)
    
    if prediction.shape == (1, 1):
        next_week_passing_yards = prediction[0][0]
        print(f"\nPredicted next week's passing yards: {next_week_passing_yards}")
    else:
        print(f"\nUnexpected prediction shape: {prediction.shape}")
        print(f"Prediction content: {prediction}")
        
except Exception as e:
    print(f"\nAn error occurred during prediction: {e}")

if next_week_passing_yards is None:
    print("\nFailed to generate a valid prediction.")

Player: P.Mahomes (ID: 00-0033873)
Position: 4
Team: 15
Current week stats:
  Passing yards: 4.503370548490142
  Rushing yards: -0.28853030402544877
  Receiving yards: -0.8017950927505366
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step

Predicted next week's passing yards: 265.2552490234375
